In [2]:
import bpnetlite

In [3]:
import torch

from bpnetlite.io import extract_peaks
from bpnetlite.io import PeakGenerator
from bpnetlite import BPNet

peaks = 'test/CTCF.peaks.bed'
seqs = '../../oak/common/hg38/hg38.fa'
signals = ['test/CTCF.plus.bw', 'test/CTCF.minus.bw']
controls = ['test/CTCF.plus.ctl.bw', 'test/CTCF.minus.ctl.bw']

training_chroms = ['chr{}'.format(i) for i in range(1, 17)]
valid_chroms = ['chr{}'.format(i) for i in range(18, 23)]

In [ ]:
training_data = PeakGenerator(peaks, seqs, signals, controls, chroms=training_chroms)

In [ ]:
X_valid, y_valid, X_ctl_valid = extract_peaks(peaks, seqs, signals, controls, chroms=valid_chroms, max_jitter=0)